---
Name: Dean Tahory 319045837,  Omer Mezig 204614770
---

Since the HW grading is done in a semi-automatic manner, please adhere to the following naming format for your submission.
Each group of students (mostly pairs, with some approved exceptions) should submit a Zip file, whose name is the underscored-separated id list of all the submitters. For example, for two submitters, the naming format is: id1_id2.zip.
The ZIP file should contain:
1. Jupyter notebook (.ipynb file and not a .zip file) whose name is the underscored-separated id list of all the submitters. For example, for two submitters, the naming format is: id1_id2.ipynb.
2. Python File (map-reduce part, .py file) whose name is the underscored-separated id list of all the submitters. For example, for two submitters, the naming format is: id1_id2.py.

In [194]:
# install pyspark library (via pip install)
# There is no need to set a local spark server. 

In [195]:
# run this command to make sure the versions of pyspark and your python match:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

for each of the tasks bellow, make sure you add a print command at the end of the cell, to show the results (df.show(1))

# Getting started:

start a spark session

In [196]:
from pyspark.sql import SparkSession
name = 'DeanAndOmer' #give a name to your app
spark = SparkSession.builder.appName('{name}'.format(name=name)).getOrCreate()

load the wine dataset: 

In [197]:
from sklearn.datasets import load_wine

wine = load_wine()
data = wine.data
columns = list(wine.feature_names)

create spark dataframe to contain the data and the columns names as it's schema

In [198]:
df = spark.createDataFrame(data= data, schema= columns)

preform 3 initial data exploration tasks that will help you understand your data better 

In [199]:
df.show()

+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+
|alcohol|malic_acid| ash|alcalinity_of_ash|magnesium|total_phenols|flavanoids|nonflavanoid_phenols|proanthocyanins|color_intensity| hue|od280/od315_of_diluted_wines|proline|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|                        3.92| 1065.0|
|   13.2|      1.78|2.14|             11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|                         3.4| 1050.0|
|  13.16|      2.36|2.67|             18.6|    101.0|          2.8|      3.24|                 0.3|           2.81|           5.68

In [200]:
df.printSchema()

root
 |-- alcohol: double (nullable = true)
 |-- malic_acid: double (nullable = true)
 |-- ash: double (nullable = true)
 |-- alcalinity_of_ash: double (nullable = true)
 |-- magnesium: double (nullable = true)
 |-- total_phenols: double (nullable = true)
 |-- flavanoids: double (nullable = true)
 |-- nonflavanoid_phenols: double (nullable = true)
 |-- proanthocyanins: double (nullable = true)
 |-- color_intensity: double (nullable = true)
 |-- hue: double (nullable = true)
 |-- od280/od315_of_diluted_wines: double (nullable = true)
 |-- proline: double (nullable = true)



In [201]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+----------------------------+-----------------+
|summary|           alcohol|        malic_acid|               ash| alcalinity_of_ash|         magnesium|     total_phenols|        flavanoids|nonflavanoid_phenols|   proanthocyanins|   color_intensity|                hue|od280/od315_of_diluted_wines|          proline|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+----------------------------+-----------------+
|  count|               178|               178|               178|               178|               178|               178|               178|                 178|               178|           

# Data preparation tasks:

preform the following data cleaning task:

    1. rename the column "Alcohol" to "ABV"


In [202]:
df.withColumnRenamed("Alcohol","ABV").show()

+-----+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+
|  ABV|malic_acid| ash|alcalinity_of_ash|magnesium|total_phenols|flavanoids|nonflavanoid_phenols|proanthocyanins|color_intensity| hue|od280/od315_of_diluted_wines|proline|
+-----+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+
|14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|                        3.92| 1065.0|
| 13.2|      1.78|2.14|             11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|                         3.4| 1050.0|
|13.16|      2.36|2.67|             18.6|    101.0|          2.8|      3.24|                 0.3|           2.81|           5.68|1.03|      

    2. create a new column that will we be the standardization of column "magnesium". call it "magnesium_stand"

In [203]:
df.describe(['magnesium']).show()
mean = df.describe(['magnesium']).collect()[1][1]
std = df.describe(['magnesium']).collect()[2][1]
df = df.withColumn("magnesium_stand",(df.magnesium/mean))
df.show()

+-------+------------------+
|summary|         magnesium|
+-------+------------------+
|  count|               178|
|   mean| 99.74157303370787|
| stddev|14.282483515295665|
|    min|              70.0|
|    max|             162.0|
+-------+------------------+

+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------------------+
|alcohol|malic_acid| ash|alcalinity_of_ash|magnesium|total_phenols|flavanoids|nonflavanoid_phenols|proanthocyanins|color_intensity| hue|od280/od315_of_diluted_wines|proline|   magnesium_stand|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------------------+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|                       

    3. create a new column that will be valued 1 (and 0 otherwise) if the alcohol rate exceeds 14 (including) and color_intensity exceeds 6. call it "col_intense".

In [204]:
from pyspark.sql.types import LongType
from pyspark.sql.functions import udf
def AlcoholAndColorFilter(alcohol,color):
    if alcohol >=14 and color >=6: return 1
    else: return 0
AlcoholAndColorFilter_udf = udf(AlcoholAndColorFilter, LongType())
df = df.withColumn("col_intense",AlcoholAndColorFilter_udf("alcohol","color_intensity"))
df.show()


+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------------------+-----------+
|alcohol|malic_acid| ash|alcalinity_of_ash|magnesium|total_phenols|flavanoids|nonflavanoid_phenols|proanthocyanins|color_intensity| hue|od280/od315_of_diluted_wines|proline|   magnesium_stand|col_intense|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------------------+-----------+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|                        3.92| 1065.0|1.2732905260786302|          0|
|   13.2|      1.78|2.14|             11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|                         3.4| 1050.0|1.00259

We will load the labels of the data above:

In [205]:
target = wine.target
column = ['Class']

create spark dataframe to contain the data and the columns of the target

In [206]:
df_target = spark.createDataFrame(data= target, schema= column)
df_target.show()

+-----+
|Class|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
+-----+
only showing top 20 rows



In [207]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+----------------------------+-----------------+-------------------+--------------------+
|summary|           alcohol|        malic_acid|               ash| alcalinity_of_ash|         magnesium|     total_phenols|        flavanoids|nonflavanoid_phenols|   proanthocyanins|   color_intensity|                hue|od280/od315_of_diluted_wines|          proline|    magnesium_stand|         col_intense|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+----------------------------+-----------------+-------------------+--------------------+
|  count|               178|               178|               178|    

merge the two datasets to one dataset

In [208]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("join_id", monotonically_increasing_id())
df_target = df_target.withColumn("join_id", monotonically_increasing_id())
merged_df = df.join(df_target, on="join_id", how="inner").drop("join_id")
merged_df.show()
df = merged_df


+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------------------+-----------+-----+
|alcohol|malic_acid| ash|alcalinity_of_ash|magnesium|total_phenols|flavanoids|nonflavanoid_phenols|proanthocyanins|color_intensity| hue|od280/od315_of_diluted_wines|proline|   magnesium_stand|col_intense|Class|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------------------+-----------+-----+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|                        3.92| 1065.0|1.2732905260786302|          0|    0|
|  14.06|      2.15|2.61|             17.6|    121.0|          2.6|      2.51|                0.31|           1.25|           5.05|1.06|                    

# Explore your data and labels:

preform exploration tasks to find the answer to each question below:

1. how many samples do you have for each class

In [209]:
# we need to filter based on the target column and find the len
df.groupBy('Class').count().show()

+-----+-----+
|Class|count|
+-----+-----+
|    0|   59|
|    1|   71|
|    2|   48|
+-----+-----+



2. which class has the highest alcohol value? which class has highest alcohol values on average?  

In [210]:
df.groupBy('Class').agg({'alcohol': 'mean'}).show()
df.groupBy('Class').agg({'alcohol': 'max'}).show()
print("Class 0 has the highest alcohol value and also in average")

+-----+------------------+
|Class|      avg(alcohol)|
+-----+------------------+
|    0|13.744745762711863|
|    1|12.278732394366196|
|    2|13.153750000000002|
+-----+------------------+

+-----+------------+
|Class|max(alcohol)|
+-----+------------+
|    0|       14.83|
|    1|       13.86|
|    2|       14.34|
+-----+------------+

Class 0 has the highest alcohol value and also in average


3. which column has the highset correlation with Class column?

In [211]:
from pyspark.sql.functions import corr
max_col_cor = 0
max_col_name = ""
for col in df.columns:
    if(col != 'Class'):
        curr_corr =  df.stat.corr('Class',col)
        if curr_corr > max_col_cor:
            max_col_cor = curr_corr
            max_col_name = col
print(f"The column with the highset correlation with Class column is: {max_col_name}")
print(f"The correlation is {max_col_cor}")

The column with the highset correlation with Class column is: alcalinity_of_ash
The correlation is 0.5178591098214529


# Train an ML model to predict the class (Bonus)

1. create a column for the featers
2. split your data to train and test (70%,30%)
3. train Random Forest model to predict the column "Class"
4. print your test results (using metric accuracy)